# VMware Documentation Analysis Experiment

이 노트북은 단일 쿼리에 대한 모델의 분석 결과를 실행하고 시각화합니다.

In [14]:
import os
import pandas as pd
import subprocess
from IPython.display import display, Markdown

# 프로젝트 루트 경로 확인
project_root = os.getcwd()
log_path = os.path.join(project_root, "results", "experiment_log.csv")

print(f"현재 작업 디렉토리: {project_root}")
print(f"로그 파일 경로: {log_path}")

현재 작업 디렉토리: /home/remote/jwoo/THU/Capstone
로그 파일 경로: /home/remote/jwoo/THU/Capstone/results/experiment_log.csv


In [15]:
def run_experiment_script():
    # 현재 환경 변수 가져오기
    env = os.environ.copy()
    # PYTHONPATH에 현재 작업 디렉토리(프로젝트 루트) 추가
    env["PYTHONPATH"] = os.getcwd()
    
    # 파이썬 실행 경로 설정
    cmd = ["python", "src/main.py"]
    
    print("🚀 실험 스크립트 실행 중 (PYTHONPATH 설정 완료)...\n")
    
    # 프로세스 시작
    process = subprocess.Popen(
        cmd, 
        stdout=subprocess.PIPE, 
        stderr=subprocess.STDOUT, 
        text=True, 
        encoding='utf-8',
        env=env # 수정된 환경 변수 전달
    )

    # 실시간 로그 출력
    if process.stdout:
        for line in iter(process.stdout.readline, ""):
            print(line, end="")
    
    process.wait()
    
    if process.returncode == 0:
        print("\n✅ 실험이 성공적으로 완료되었습니다.")
    else:
        print(f"\n❌ 실험 중 오류가 발생했습니다. (Return Code: {process.returncode})")

# 실험 실행
run_experiment_script()

🚀 실험 스크립트 실행 중 (PYTHONPATH 설정 완료)...

=== Long Context LLM Experiment Runner (v4.5) ===

[실험 시작] Experiment: VMware Documentation
[*] 대상 파일: vmware.txt

  [질문 처리 중] List the differences of the 3q, 3s and 3m release in a table...

[RUN] 파이프라인 실행 시작
쿼리: List the differences of the 3q, 3s and 3m release in a table.

[STEP 1/8] 데이터 전처리 및 캐시 확인 중...
    >>> 파일 분석 중: vmware.txt

Chunking: 100%|██████████| 1/1 [00:00<00:00,  1.03it/s]
[DONE] 전처리 완료 (2.57s)

[STEP 2/8] 벡터 임베딩 생성 및 동적 전략 수립 중...
    >>> [임베딩 생성] 2318개 청크에 대한 벡터화 진행 중...

Batches: 100%|██████████| 10/10 [01:21<00:00,  8.15s/it]
    >>> 동적 전략 결과: 탐색 범위(N)=120, 추출 범위(K)=30
[DONE] 전략 수립 완료 (85.08s)

[STEP 3/8] 벡터 유사도 기반 후보 청크 탐색 중...
    >>> 전체 2318개 중 130개의 후보군 선별 완료
[DONE] 후보 탐색 완료 (0.26s)

[STEP 4/8] 개별 청크 정밀 분석(Scouting) 진행 중 (병렬 처리)...

Scouting: 100%|██████████| 130/130 [00:05<00:00, 22.84it/s]
[DONE] 정밀 분석 완료 (5.75s)

[STEP 5/8] 고가치 정보 선별 및 증거 패키징 중...
    >>> 상위 30개의 증거가 최종 합성에 포함됩니다.
[DONE] 패키징 완료 (0.00s)

[STEP 6/8] 기술 보고

In [38]:
if os.path.exists(log_path):
    # CSV 로드 (utf-8-sig 인코딩 적용)
    df = pd.read_csv(log_path)
    
    display(Markdown("## 📊 Experiment Summary"))
    
    # 기본 통계 및 테이블 출력
    display(df.tail(10)) # 최근 실행 결과 10건
    
    # 실험별 평균 소요 시간 계산
    if 'latency' in df.columns:
        avg_latency = df.groupby('experiment_name')['latency'].apply(
            lambda x: x.str.replace('s', '').astype(float).mean()
        )
        display(Markdown("### 🕒 Experiment Inference Time (Latency)"))
        print(avg_latency)
else:
    print("⚠️ 결과 로그 파일을 찾을 수 없습니다. 실험이 정상적으로 완료되었는지 확인하세요.")

## 📊 Experiment Summary

,timestamp,experiment_name,query,answer,latency,source_files
0,2026-02-11 11:01:32,VMware Documentation,"List the differences of the 3q, 3s and 3m rele...","Based on the provided evidence, the difference...",212.89s,vmware.txt
1,2026-02-11 11:01:54,VMware Documentation,List me a timeline for when each version of th...,"Based on the provided evidence, here is a time...",21.87s,vmware.txt
2,2026-02-11 11:02:52,EU AI Act Analysis,How many articles are there in the AI Act?,The AI Act contains a total of 103 articles.,55.62s,aiact.txt
3,2026-02-11 11:03:13,EU AI Act Analysis,Summarize the requirements for high-risk AI sy...,The requirements for high-risk AI systems are ...,20.62s,aiact.txt
4,2026-02-11 11:03:30,EU AI Act Analysis,What are the prohibited AI practices according...,The prohibited AI practices according to Artic...,16.8s,aiact.txt
5,2026-02-11 11:04:38,EU National Parks File,List me all national parks sorted by their hig...,"Based on the provided evidence, the national p...",65.51s,nationalparks_europe.txt
6,2026-02-11 11:05:22,EU National Parks File,List me all national parks where I can see bro...,"Based on the provided evidence, the following ...",43.99s,nationalparks_europe.txt
7,2026-02-11 11:05:36,EU National Parks File,Give me a table of all the national parks in s...,"Based on the provided evidence, the national p...",14.32s,nationalparks_europe.txt
8,2026-02-11 11:06:50,AI Act + Top generative use case,Given a virtual ai assistant in the healthcare...,The AI Act regulations relevant to a virtual A...,71.56s,"aiact.txt, top_generative_use_cases.txt"
9,reranker model (cohere reranker),NaN,NaN,NaN,NaN,NaN


### 🕒 Experiment Inference Time (Latency)

experiment_name
AI Act + Top generative use case     71.560000
EU AI Act Analysis                   31.013333
EU National Parks File               41.273333
VMware Documentation                117.380000
Name: latency, dtype: float64


In [34]:
if os.path.exists(log_path):
    df = pd.read_csv(log_path)
    if not df.empty:
        latest_exp = df['experiment_name'].iloc[0]
        display(Markdown(f"### 🔍 Experiment Output: **{latest_exp}**"))
        
        subset = df[df['experiment_name'] == latest_exp]
        for _, row in subset.iterrows():
            display(Markdown(f"---"))
            display(Markdown(f"**Query:** {row['query']}"))
            display(Markdown(f"**Answer:** {row['answer']}"))
            display(Markdown(f"*InferTime: {row.get('latency', 'N/A')} | Source: {row.get('source_files', 'N/A')}*"))


### 🔍 Experiment Output: **VMware Documentation**

---

**Query:** List the differences of the 3q, 3s and 3m release in a table.

**Answer:** Based on the provided evidence, the differences between the 3q, 3s, and 3m releases are as follows:  ### ESXi 7.0 Update 3q - **Build Details**:   - Download Filename: VMware-ESXi-7.0U3q-21686933-depot.zip   - Build: 21686933   - Download Size: 379.4 MB   - md5sum: 9cd52571752b939637f80b351daace94   - sha256checksum: 8c7892af03254259db28e7f2982be2802dcf66bdb2fc92e15a06fe7 e7835b692   - Host Reboot Required: Yes   - Virtual Machine Migration or Shutdown Required: Yes  ### ESXi 7.0 Update 3s - **Build Details**:   - Download Filename: VMware-ESXi-7.0U3s-21686933-depot.zip   - Build: 21686933   - Download Size: 379.4 MB   - md5sum: 9cd52571752b939637f80b351daace94   - sha256checksum: 8c7892af03254259db28e7f2982be2802dcf66bdb2fc92e15a06fe7 e7835b692   - Host Reboot Required: Yes   - Virtual Machine Migration or Shutdown Required: Yes  ### ESXi 7.0 Update 3m - **Build Details**:   - Download Filename: VMware-ESXi-7.0U3m-21686933-depot.zip   - Build: 21686933   - Download Size: 379.4 MB   - md5sum: 9cd52571752b939637f80b351daace94   - sha256checksum: 8c7892af03254259db28e7f2982be2802dcf66bdb2fc92e15a06fe7 e7835b692   - Host Reboot Required: Yes   - Virtual Machine Migration or Shutdown Required: Yes  ### Components - **ESXi Component - core ESXi VIBs**:   - Bulletin: ESXi_7.0.3-0.90.21686933   - Category: Bugfix   - Severity: Critical - **ESXi Install/Upgrade Component**:   - Bulletin: esx-update_7.0.3-0.90.21686933   - Category: Bugfix   - Severity: Critical - **Broadcom NetXtreme I ESX VMKAPI ethernet driver**:   - Bulletin: Broadcom-ntg3_4.1.9.0-5vmw.703.0.90.21686933   - Category: Bugfix   - Severity: Critical  ### Rollup Bulletin - **Bulletin ID**: ESXi70U3q-21686933, ESXi70U3s-21686933, ESXi70U3m-21686933 - **Category**: Bugfix - **Severity**: Critical  ### Image Profiles - **ESXi-7.0U3q-21686933-standard**:   - Build: For build information, see Patches Contained in this Release.   - Vendor: VMware, Inc.   - Release Date: May 03, 2023   - Acceptance Level: PartnerSupported   - Affected Hardware: N/A   - Affected Software: N/A   - Affected VIBs: List of VIBs   - PRs Fixed: 3164897, 3182870   - Related CVE numbers: N/A - **ESXi-7.0U3q-21686933-no-tools**:   - Build: For build information, see Patches Contained in this Release.   - Vendor: VMware, Inc.   - Release Date: May 03, 2023   - Acceptance Level: PartnerSupported   - Affected Hardware: N/A   - Affected Software: N/A   - Affected VIBs: List of VIBs   - PRs Fixed: 3164897, 3182870   - Related CVE numbers: N/A  ### ESXi Image - **Name and Version**: ESXi70U3q-21686933, ESXi70U3s-21686933, ESXi70U3m-21686933 - **Release Date**: May 03, 2023 - **Category**: Bugfix - **Affected Components**:   - ESXi Component - core ESXi VIBs   - ESXi Install/Upgrade Component   - Broadcom NetXtreme I ESX VMKAPI ethernet - **PRs Fixed**: 3164897, 3182870 - **Related CVE numbers**: N/A  ### Known Issues from Previous Releases - **vSAN Issues**:   - If you change the preferred site in a VMware vSAN Stretched Cluster, some objects might incorrectly appear as compliant. - **vSphere Cluster Services Issues**:   - You see compatibility issues in new vCLS VMs deployed in vSphere 7.0 Update 3 environment. - **Installation, Upgrade, and Migration Issues**:   - The vCenter Upgrade/Migration pre-checks fail with "Unexpected error 87".   - Problems upgrading to vSphere 7.0 with pre-existing CIM providers.   - Installation of 7.0 Update 1 drivers on ESXi 7.0 hosts might fail.   - UEFI booting of ESXi hosts might stop with an error during an update to ESXi 7.0 Update 2 from an earlier version of ESXi 7.0.   - If legacy VIBs are in use on an ESXi host, vSphere Lifecycle Manager cannot extract a desired software specification to seed to a new cluster.   - You see a short burst of log messages in the syslog.log after every ESXi boot.   - You see warning messages for missing VIBs in vSphere Quick Boot compatibility check reports.   - Auto bootstrapping a cluster that you manage with a vSphere Lifecycle Manager image fails with an error.   - Upgrades to ESXi 7.x from 6.5.x and 6.7.0 by using ESXCLI might fail due to a space limitation.  ### Resolved Issues - **ESXi_7.0.3-0.90.21686933**:   - **Patch Category**: Bugfix   - **Patch Severity**: Critical   - **Host Reboot Required**: Yes   - **Virtual Machine Migration or Shutdown Required**: Yes   - **Affected Hardware**: N/A   - **Affected Software**: N/A   - **Affected VIBs**: List of VIBs   - **PRs Fixed**: 3164897   - **CVE numbers**: N/A   - **Description**: This patch updates the esx-base VIB. Due to their dependency with the esx-base VIB, the following VIBs are updated with build number and patch version changes, but deliver no fixes: gc, esx-dvfilter-generic-fastpath, native-miscdrivers, bmcal, esx-xserver, vsanhealth, vdfs, cpu-microcode, vsan, esxio-combiner, crx, and trx. This patch resolves the following issue:     - PR 3164897: After an upgrade to ESXi 7.0 Update 3l, some ESXi hosts and virtual machines connected to virtual switches might lose network. - **esx-update_7.0.3-0.90.21686933**:   - **Patch Category**: Bugfix   - **Patch Severity**: Critical   - **Host Reboot Required**: Yes   - **Virtual Machine Migration or Shutdown Required**: Yes   - **Affected Hardware**: N/A   - **Affected Software**: N/A   - **Affected VIBs**: loadesx and esx-update.   - **PRs Fixed**: N/A   - **CVE numbers**: N/A   - **Description**: Due to their dependency with the esx-base VIB, the following VIBs are updated with build number and patch version changes, but deliver no fixes: loadesx and esx-update. - **Broadcom-ntg3_4.1.9.0-5vmw.703.0.90.21686933**:   - **Patch Category**: Bugfix   - **Patch Severity**: Critical   - **Host Reboot Required**: Yes   - **Virtual Machine Migration or Shutdown Required**: Yes   - **Affected Hardware**: N/A   - **Affected Software**: N/A   - **Affected VIBs**: VMW_bootbank_ntg3_4.1.9.0-5vmw.703.0.90.21686933   - **PRs Fixed**: 3182870   - **CVE numbers**: N/A   - **Description**: Updates the ntg3 VIB to resolve the following issue:     - PR 3182870: After upgrading the ntg3 driver to version 4.1.9.0-4vmw, Broadcom NICs with fiber physical connectivity might lose network.  ### Summary Table | Release | Build | Download Size | md5sum | sha256checksum | Host Reboot Required | Virtual Machine Migration or Shutdown Required | |---------|-------|---------------|---------|----------------|----------------------|-----------------------------------------------| | 3q      | 21686933 | 379.4 MB | 9cd52571752b939637f80b351daace94 | 8c7892af03254259db28e7f2982be2802dcf66bdb2fc92e15a0

*InferTime: 212.89s | Source: vmware.txt*

---

**Query:** List me a timeline for when each version of the Vmware vCenter was released.

**Answer:** Based on the provided evidence, here is a timeline of VMware vCenter Server versions and their release dates:  | Version | Release Date | |---------|--------------| | vCenter Server 7.0 | Not specified in the evidence | | vCenter Server 7.0.0a | Not specified in the evidence | | vCenter Server 7.0.0b | Not specified in the evidence | | vCenter Server 7.0.0c | Not specified in the evidence | | vCenter Server 7.0.0d | Not specified in the evidence | | vCenter Server 7.0 Update 1 | Not specified in the evidence | | vCenter Server 7.0 Update 1a | Not specified in the evidence | | vCenter Server 7.0 Update 1b | Not specified in the evidence | | vCenter Server 7.0 Update 1c | Not specified in the evidence | | vCenter Server 7.0 Update 1d | Not specified in the evidence | | vCenter Server 7.0 Update 2 | Not specified in the evidence | | vCenter Server 7.0 Update 2a | Not specified in the evidence | | vCenter Server 7.0 Update 2b | Not specified in the evidence | | vCenter Server 7.0 Update 2c | Not specified in the evidence | | vCenter Server 7.0 Update 2d | Not specified in the evidence | | vCenter Server 7.0 Update 2e | Not specified in the evidence | | vCenter Server 7.0 Update 3 | Not specified in the evidence | | vCenter Server 7.0 Update 3a | Not specified in the evidence | | vCenter Server 7.0 Update 3c | Not specified in the evidence | | vCenter Server 7.0 Update 3d | Not specified in the evidence | | vCenter Server 7.0 Update 3e | Not specified in the evidence | | vCenter Server 7.0 Update 3f | Not specified in the evidence | | vCenter Server 7.0 Update 3g | Not specified in the evidence | | vCenter Server 7.0 Update 3i | Not specified in the evidence | | vCenter Server 7.0 Update 3j | Not specified in the evidence | | vCenter Server 7.0 Update 3k | Not specified in the evidence | | vCenter Server 7.0 Update 3l | Not specified in the evidence | | vCenter Server 7.0 Update 3m | Not specified in the evidence | | vCenter Server 7.0 Update 3n | Not specified in the evidence | | vCenter Server 7.0 Update 3o | Not specified in the evidence | | vCenter Server 7.0 Update 3p | Not specified in the evidence |  The evidence provided does not contain specific release dates for the vCenter Server versions listed. The table above includes the versions mentioned in the evidence, but their release dates are not specified.

*InferTime: 21.87s | Source: vmware.txt*